In [1]:
!pip cache purge

Files removed: 3019


In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.0 MB/s eta 0:00:00


In [3]:
!pip install torch==2.3.1
!pip install dgl==1.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.18.1
    Uninstalling nvidia-nccl-cu12-2.18.1:
      Successfully uninstalled nvidia-nccl-cu12-2.18.1
  Attempting uninstall: torch
    Found existing installation: torch 2.1.2
    Uninstalling torch-2.1.2:
      Successfully uninstalled torch-2.1.2


In [4]:
import torch

In [23]:
import dgl

In [58]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
import torch.nn as nn
import dgl.nn as dglnn
import torch.nn as nn
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [7]:
book_history = pd.read_csv('book_history.dat', sep='\t')
book_ratings = pd.read_csv('book_ratings.dat', sep='\t')
items_info = pd.read_csv('items_info.csv', sep=',').drop(columns=['Unnamed: 0', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'ISBN']).rename(columns={'Book-Title': 'Book_Title', 'Book-Author': 'Book_Author', 'Year-Of-Publication': 'Year_Of_Publication'})

In [8]:
book_history.head()

,user,item,accessed
0,1,152,1
1,1,153,1
2,1,2176,1
3,1,154,1
4,1,734,1


In [9]:
book_ratings.head()

,user,item,rating
0,1,6264,7.0
1,1,4350,7.0
2,1,6252,5.0
3,1,202,9.0
4,1,6266,6.0


In [10]:
items_info.head()

,Book_ID,Book_Title,Book_Author,Year_Of_Publication,Publisher
0,1,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
1,2,The Mummies of Urumchi,E. J. W. Barber,1999,"W. W. Norton &, Company"
2,3,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group
3,4,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume
4,5,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press


In [11]:
items_info.shape[0], items_info['Publisher'].nunique(), items_info['Year_Of_Publication'].nunique(), items_info['Book_Author'].nunique()

(17384, 2364, 69, 9646)

Проверка на дубликаты:

In [12]:
items_info = items_info.drop_duplicates(subset='Book_ID')
num_items = len(items_info)
num_items == 17384

True

Приведение текстовых данных к нижнему регистру:

In [13]:
items_info['Book_Title'] = items_info['Book_Title'].apply(lambda x: x.lower())
items_info['Book_Author'] = items_info['Book_Author'].apply(lambda x: x.lower())
items_info['Publisher'] = items_info['Publisher'].apply(lambda x: x.lower())

In [14]:
items_info.head()

,Book_ID,Book_Title,Book_Author,Year_Of_Publication,Publisher
0,1,decision in normandy,carlo d'este,1991,harperperennial
1,2,the mummies of urumchi,e. j. w. barber,1999,"w. w. norton &, company"
2,3,what if?: the world's foremost military histor...,robert cowley,2000,berkley publishing group
3,4,beloved (plume contemporary fiction),toni morrison,1994,plume
4,5,our dumb century: the onion presents 100 years...,the onion,1999,three rivers press


Проверка на пропущенные значения:

In [15]:
items_info.isna().sum()

,0
Book_ID,0
Book_Title,0
Book_Author,0
Year_Of_Publication,0
Publisher,0


Отбор пользователей с более чем 50 взаимодействиями:

In [16]:
user_counts = book_history['user'].value_counts()
selected_users = set(user_counts[user_counts >= 50].index)

Фильтрация book_history и book_ratings по выбранным пользователям:

In [17]:
book_history = book_history[book_history['user'].isin(selected_users)]
book_ratings = book_ratings[book_ratings['user'].isin(selected_users)]

Количество уникальных пользователей и книг:

In [18]:
print('Unique users:', len(book_history['user'].unique()))
print('Unique books:', len(items_info['Book_ID'].unique()))

Unique users: 1320
Unique books: 17384


Замена идентификаторов:

In [19]:
unique_users = book_history['user'].unique()
user_mapping = {user: idx for idx, user in enumerate(unique_users)}

In [20]:
unique_books = items_info['Book_ID'].unique()
book_mapping = {book_id: idx for idx, book_id in enumerate(unique_books)}

In [21]:
book_history['user'] = book_history['user'].map(user_mapping)
book_history['item'] = book_history['item'].map(book_mapping)
book_ratings['user'] = book_ratings['user'].map(user_mapping)
book_ratings['item'] = book_ratings['item'].map(book_mapping)

In [22]:
texts = items_info[['Book_Title', 'Book_Author', 'Publisher']].apply(lambda row: ' '.join(row), axis=1)
vectorizer = SentenceTransformer('all-MiniLM-L6-v2')
text_feats = vectorizer.encode(texts.tolist())

numeric_columns = ['Year_Of_Publication']
numeric_data = items_info[numeric_columns]
scaler = StandardScaler()
scaled_numeric_data = scaler.fit_transform(numeric_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
print(f"numeric_feats shape: {scaled_numeric_data.shape}")
print(f"text_feats shape: {text_feats.shape}")

numeric_feats shape: (17384, 1)
text_feats shape: (17384, 384)


Объединение признаков:

In [25]:
numeric_feats = torch.tensor(scaled_numeric_data, dtype=torch.float)
text_feats = torch.tensor(text_feats, dtype=torch.float)
book_feats = torch.cat((numeric_feats, text_feats), dim=1)

Создание графа:

In [26]:
graph_data = {
    ('user', 'rated', 'book'): (torch.tensor(book_ratings['user'].values), torch.tensor(book_ratings['item'].values)),
    ('book', 'rated-by', 'user'): (torch.tensor(book_ratings['item'].values), torch.tensor(book_ratings['user'].values))
}
g = dgl.heterograph(graph_data)

Разделение на обучающую и тестовую выборки:

In [27]:
user_ids, book_ids = g.edges(etype='rated')
n_edges = user_ids.shape[0]
while True:
    train_ids = np.random.choice(np.arange(n_edges), size=int(n_edges * 0.8), replace=False)
    test_ids = torch.tensor(list(set(np.arange(n_edges)) - set(train_ids))).long()
    train_ids = torch.tensor(train_ids).long()
    if not set(user_ids[test_ids].tolist()) - set(user_ids[train_ids].tolist()):
        break
target_ratings = torch.tensor(book_ratings['rating'].values, dtype=torch.float)
train_ratings = target_ratings[train_ids]
test_ratings = target_ratings[test_ids]

In [28]:
print(f'Количество связей в обучающей выборке: {train_ids.shape[0]}')
print(f'Количество связей в тестовой выборке: {test_ids.shape[0]}')

Количество связей в обучающей выборке: 43333
Количество связей в тестовой выборке: 10834


In [102]:
class RatingPredictor(nn.Module):
    def __init__(self, book_dim, hidden_dim, user_dim, out_dim):
        super(RatingPredictor, self).__init__()
        self.user_emb = nn.Embedding(num_embeddings=g.num_nodes('user'), embedding_dim=user_dim)
        self.book_emb = nn.Embedding(num_embeddings=g.num_nodes('book'), embedding_dim=book_dim)
        self.layer1 = dglnn.HeteroGraphConv({
            'rated': dglnn.GraphConv(in_feats=user_dim, out_feats=hidden_dim),
            'rated-by': dglnn.GraphConv(in_feats=book_dim, out_feats=hidden_dim)
        }, aggregate='sum')
        self.layer2 = dglnn.HeteroGraphConv({
            'rated': dglnn.GraphConv(in_feats=hidden_dim, out_feats=out_dim),
            'rated-by': dglnn.GraphConv(in_feats=hidden_dim, out_feats=out_dim)
        }, aggregate='sum')
        self.output_layer = nn.Linear(2 * out_dim, 1)

    def forward(self, graph, book_features, edge_indices):
        user_features = self.user_emb.weight
        book_features = self.book_emb.weight

        conv1_output = self.layer1(graph, {'book': book_features, 'user': user_features})
        conv1_output['user'] = torch.relu(conv1_output['user'])
        conv1_output['book'] = torch.relu(conv1_output['book'])
        conv2_output = self.layer2(graph, conv1_output)

        u_ids, b_ids = graph.edges(etype='rated')
        u_ids, b_ids = u_ids[edge_indices], b_ids[edge_indices]

        combined_features = torch.cat([conv2_output['user'][u_ids], conv2_output['book'][b_ids]], dim=1)
        rating_prediction = self.output_layer(combined_features)

        return rating_prediction

In [103]:
book_feat_dim = book_feats.shape[1]
hidden_feats = 128
user_feat_dim = 256
output_dim = 256

In [104]:
model = RatingPredictor(book_feat_dim, hidden_feats, user_feat_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_loss = []
test_loss = []
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    user_ratings_pred = model(g, book_feats, train_ids)
    loss = criterion(user_ratings_pred.squeeze(), target_ratings[train_ids])
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

    if epoch % 10 == 0:
        model.eval()
        user_ratings_pred = model(g, book_feats, test_ids)
        loss = criterion(user_ratings_pred.squeeze(), target_ratings[test_ids])
        print(f'Epoch [{epoch+1}/{num_epochs}], Val loss: {loss.item():.4f}')
        test_loss.append(loss.item())

Epoch [1/100], Val loss: 61.1429
Epoch [11/100], Val loss: 14.7313
Epoch [21/100], Val loss: 10.9041
Epoch [31/100], Val loss: 7.0076
Epoch [41/100], Val loss: 5.8257
Epoch [51/100], Val loss: 5.5236
Epoch [61/100], Val loss: 5.1413
Epoch [71/100], Val loss: 4.8534
Epoch [81/100], Val loss: 4.5627
Epoch [91/100], Val loss: 4.3302


Получение эмбеддингов пользователей и книг:

In [46]:
user_embeddings = model.user_emb.weight.detach().cpu().numpy()
book_embeddings = model.book_emb.weight.detach().cpu().numpy()
print(f"User embeddings shape: {user_embeddings.shape}")
print(f"Book embeddings shape: {book_embeddings.shape}")

User embeddings shape: (1320, 256)
Book embeddings shape: (17384, 385)


Рекомендательная система:

In [101]:
cos_sim_matrix = np.abs(cosine_similarity(user_embeddings))
print(f"Cosine similarity matrix shape: {cos_sim_matrix.shape}")

Cosine similarity matrix shape: (1320, 1320)


In [57]:
def get_recommendations(user_id, user_to_index, index_to_user, cos_sim_matrix, ratings, n_user_for_rec=5):
    """
    Функция для получения рекомендаций для пользователя на основе матрицы косинусного сходства.

    Parameters:
    - user_id: ID пользователя, для которого нужно получить рекомендации.
    - user_to_index: Словарь для преобразования идентификаторов пользователей в индексы.
    - index_to_user: Словарь для преобразования индексов в идентификаторы пользователей.
    - cos_sim_matrix: Матрица косинусного сходства между пользователями.
    - ratings: DataFrame с оценками (включает 'user', 'item', 'rating').
    - n_user_for_rec: Количество похожих пользователей для получения рекомендаций.

    Returns:
    - recommendations: DataFrame с рекомендованными книгами и их оценками.
    """
    user_index = user_to_index[user_id]

    if user_index >= len(cos_sim_matrix):
        print(f"Index {user_index} is out of bounds for cosine similarity matrix.")
        return pd.DataFrame(columns=['item', 'predicted_rating'])

    user_similarities = cos_sim_matrix[user_index]
    similar_users_indices = user_similarities.argsort()[-n_user_for_rec:][::-1]
    similar_users_ids = [index_to_user.get(idx, None) for idx in similar_users_indices if idx in index_to_user]

    if not similar_users_ids:
        print(f"No similar users found for user {user_id}.")
        return pd.DataFrame(columns=['item', 'predicted_rating'])

    similar_users_ratings = ratings[ratings['user'].isin(similar_users_ids)]
    recommendations = similar_users_ratings.groupby('item')['rating'].mean().reset_index()
    recommendations.columns = ['item', 'predicted_rating']

    return recommendations

In [61]:
def evaluate_recommendations(ratings, cos_sim_matrix, test_ratings, user_to_index, index_to_user, n_values=[5, 10, 15, 20, 25, 30]):
    """
    Оценка рекомендаций по метрикам MAE, MSE, R2.

    Parameters:
    - ratings: DataFrame с оценками (включает 'user', 'item', 'rating').
    - cos_sim_matrix: Матрица косинусного сходства между пользователями.
    - test_ratings: DataFrame с тестовыми оценками (включает 'user', 'item', 'rating').
    - user_to_index: Словарь для преобразования идентификаторов пользователей в индексы.
    - index_to_user: Словарь для преобразования индексов в идентификаторы пользователей.
    - n_values: Список значений n (количество похожих пользователей) для оценки.

    """
    for n in n_values:
        mae_list = []
        mse_list = []
        r2_list = []

        for user_id in ratings['user'].unique():
            recommendations = get_recommendations(user_id, user_to_index, index_to_user, cos_sim_matrix, ratings, n_user_for_rec=n)
            true_ratings = test_ratings[test_ratings['user'] == user_id]
            recommended_books = recommendations['item']
            true_ratings_for_recommendations = true_ratings[true_ratings['item'].isin(recommended_books)]

            if true_ratings_for_recommendations.empty:
                continue

            merged = recommendations.merge(true_ratings_for_recommendations, on='item', how='inner')

            if 'predicted_rating' in merged.columns and 'rating' in merged.columns:
                y_true = merged['rating']
                y_pred = merged['predicted_rating']

                if len(y_true) > 0:
                    mae = mean_absolute_error(y_true, y_pred)
                    mse = mean_squared_error(y_true, y_pred)
                    r2 = r2_score(y_true, y_pred)

                    mae_list.append(mae)
                    mse_list.append(mse)
                    r2_list.append(r2)

        if len(mae_list) > 0:
            mean_mae = np.mean(mae_list)
            mean_mse = np.mean(mse_list)
            mean_r2 = np.mean(r2_list)
        else:
            mean_mae = np.nan
            mean_mse = np.nan
            mean_r2 = np.nan

        print(f'Количество похожих пользователей: {n}')
        print(f'Средняя MAE = {mean_mae}')
        print(f'Средняя MSE = {mean_mse}')
        print(f'Средняя R2 = {mean_r2}')
        print()

In [62]:
# Создание словарей преобразования
unique_users = book_ratings['user'].unique()
user_to_index = {user: idx for idx, user in enumerate(unique_users)}
index_to_user = {idx: user for user, idx in user_to_index.items()}

evaluate_recommendations(book_ratings, cos_sim_matrix, book_ratings, user_to_index, index_to_user)

Количество похожих пользователей: 5
Средняя MAE = 0.025615875451128072
Средняя MSE = 0.03776753150755634
Средняя R2 = 0.977222700442851

Количество похожих пользователей: 10
Средняя MAE = 0.05494888486283474
Средняя MSE = 0.08191191677747825
Средняя R2 = 0.9539564231619925

Количество похожих пользователей: 15
Средняя MAE = 0.08233572387120922
Средняя MSE = 0.12350874247564018
Средняя R2 = 0.9301548417634246

Количество похожих пользователей: 20
Средняя MAE = 0.10777313657407452
Средняя MSE = 0.1631122921526466
Средняя R2 = 0.909306085378313

Количество похожих пользователей: 25
Средняя MAE = 0.1316374172420652
Средняя MSE = 0.2014678456778489
Средняя R2 = 0.8889691503378506

Количество похожих пользователей: 30
Средняя MAE = 0.15438144990797506
Средняя MSE = 0.23876344233200866
Средняя R2 = 0.8699245774406308



Рекомендация для конкретного пользователя:

In [63]:
user_id = 50
recommended_books_for_user = get_recommendations(user_id, user_to_index, index_to_user, cos_sim_matrix, book_ratings, n_user_for_rec=5)
print(f"Recommended books for user {user_id}: {recommended_books_for_user}")

Recommended books for user 50:       item  predicted_rating
0        3               8.0
1       13               8.0
2       31               7.0
3       55               9.0
4       65               7.0
..     ...               ...
310  16770               9.0
311  16813              10.0
312  16851              10.0
313  16854              10.0
314  16869               8.0

[315 rows x 2 columns]


In [64]:
rating_threshold = 8.0
book_index_to_title = dict(zip(items_info['Book_ID'], items_info['Book_Title']))
high_rating_recommendations = recommended_books_for_user[recommended_books_for_user['predicted_rating'] >= rating_threshold]
high_rating_recommendations = high_rating_recommendations.sort_values(by='predicted_rating', ascending=False)
high_rating_recommendations['item'] = high_rating_recommendations['item'].map(book_index_to_title)
print(high_rating_recommendations)

                                                  item  predicted_rating
226       the magic of you (malory novels (paperback))              10.0
300                                         the tattoo              10.0
140  the short happy life of the brown oxford: and ...              10.0
41                                    carnal innocence              10.0
197                                        the promise              10.0
..                                                 ...               ...
1                                to kill a mockingbird               8.0
170                                      separate beds               8.0
172     man at work (avon light contemporary romances)               8.0
173                                  the sands of time               8.0
314       dylan's last dare (silhouette romance, 1711)               8.0

[227 rows x 2 columns]
